In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchtext
import random
import math
import time
import torch.nn.functional as F

from tqdm.notebook import tqdm
from torch import optim
from torch.utils.data import DataLoader, Dataset
import numpy as np
np.random.seed(1)
torch.manual_seed(1)
random.seed(1)

In [ ]:
import os
os.chdir("/content/drive/MyDrive/MADE/Project/deap")

In [ ]:
def get_padding(in_size, kernel_size, stride):
    if (in_size % stride == 0):
        padding = max(kernel_size - stride, 0)
    else:
        padding = max(kernel_size - (in_size % stride), 0)
    return (padding)    

In [ ]:
def get_temporal_feature_extractor(input_size):
  pad = get_padding(input_size, 5,  2)
  conv1 = nn.Conv3d(1, 32, kernel_size = (1, 1, 5), stride=(1, 1, 2), padding=(0, 0, pad))
  relu1 = nn.ReLU()
  pad = get_padding(input_size, 3,  2)
  conv2 = nn.Conv3d(32, 32, kernel_size = (1, 1, 3), stride=(1, 1, 2), padding=(0, 0, pad))
  relu2 = nn.ReLU()
  conv3 = nn.Conv3d(32, 32, kernel_size = (1, 1, 3), stride=(1, 1, 2), padding=(0, 0, pad))
  relu3 = nn.ReLU()
  conv4 = nn.Conv3d(32, 32, kernel_size = (1, 1, 16), stride=(1, 1, 16), padding=0)
  relu4 = nn.ReLU()
  #print("11")
  result = torch.nn.Sequential(conv1, relu1, conv2, relu2, conv3, relu3, conv4, relu4)
  #print("22")
  #print(result)
  return(result)

In [ ]:
def get_regional_feature_extractor():
  conv1 = nn.Conv2d(32, 32, kernel_size = (3, 3), stride=1 , padding='same')
  relu1 = nn.ReLU()
  conv2 = nn.Conv2d(32, 32, kernel_size = (3, 3), stride=1 , padding='same')
  relu2 = nn.ReLU()
  return(torch.nn.Sequential(conv1, relu1, conv2, relu2))

In [ ]:
class Asymmetric_feature_extractor(torch.nn.Module): 
   def __init__(self):
     super().__init__()
     self.conv = nn.Conv2d(32, 64, kernel_size = 1, stride=1 , padding='same')
     self.relu = nn.ReLU()
   def forward(self, input):
     #input(bs, h, w, nf)
     half_mat = torch.split(input, (4, 1, 4), dim = 2)
     #print(half_mat.shape)
     input_new =  half_mat[0] -  half_mat[2]
     #print(input.shape)
     output = self.conv(input_new)
     output = self.relu(output)
     #print(output.shape)
     return output

def get_asymmetric_feature_extractor():
    return (Asymmetric_feature_extractor())


In [ ]:
class EmotionNet(torch.nn.Module): 
   def __init__(self, hcanals, wcanals, nfeatures, ntimes_in_sample):
      super().__init__()
      #print("1")
      self.tfe = get_temporal_feature_extractor(ntimes_in_sample) #(bs, 1, h = 9, w = 9, s = 128) -> (bs, h = 9, w = 9, s = 1)
      #print("2")
      self.rfe = get_regional_feature_extractor() #(h = 9, w = 9, s = 32) -> (h = 9, w = 9, s = 32)
      #print("3")
      self.afe = get_asymmetric_feature_extractor()
      #print("4")
      self.flat1 = nn.Flatten(1, 3)
      self.flat2 = nn.Flatten(1, 3)
      self.input_linear_size = int(hcanals * (wcanals//2)* nfeatures * 2 + hcanals * (wcanals)* nfeatures)
      #print(self.input_linear_size)
      #print("5")
      self.fc1 = nn.Linear(self.input_linear_size, 20)
      self.relu1 = nn.ReLU()
      self.drop = nn.Dropout(0.3)
      self.fc2 = nn.Linear(20, 2)
   def forward(self, input):
      input = input.unsqueeze(1)
      #print(f"input_shape = {input.shape}")
      
      #input (bs, in_canals = 1,  h=9, w=9, s=128)
      output_tfe = self.tfe(input)
      #print(f"output_tfe.shape = {output_tfe.shape}")
      #output_tfe (bs, in_canals = 32,  h=9, w=9, s=1)
      output_tfe = output_tfe.squeeze(4)
      #print(f"output_tfe.shape = {output_tfe.shape}")
      
      #output_rfe (bs, canals = 32,  h=9, w=9)
      output_rfe = self.rfe(output_tfe)
      #print(f"output_rfe.shape = {output_rfe.shape}")

      output_afe = self.afe(output_tfe)
      #print(f"output_afe.shape = {output_afe.shape}")
      #output_rfe (bs, canals = 64,  h=4, w=9)
      output_rfe_flatten = self.flat1(output_rfe)
      #print(f"output_rfe_flatten.shape = {output_rfe_flatten.shape}")
      output_afe_flatten = self.flat2(output_afe)
      #print(f"output_afe_flatten.shape = {output_afe_flatten.shape}")
      output1 = self.drop(self.fc1(torch.cat((output_rfe_flatten, output_afe_flatten), dim = 1)))
      #print(f"output1.shape = {output1.shape}")
      output1_relu = self.relu1(output1)
      #print(f"output1_relu.shape = {output1_relu.shape}")
      output2 = self.fc2(output1_relu)
      #print(f"output2.shape = {output2.shape}")
      return output2


In [ ]:
LEN_RECORD_IN_SECONDS = 60
NVIDEOS = 40
HCANALS = 9
WCANALS = 9
NTIMES_IN_SAMPLE = 128
NTIMES_IN_SEC = 128
NCANALS = 32
NFEATURES = 32
electrode_matrix = {}
electrode_matrix['FP1'] = [0, 3]
electrode_matrix['FP2'] = [0, 5]
electrode_matrix['AF3'] = [1, 3]
electrode_matrix['AF4'] = [1, 5]
electrode_matrix['F7']  = [2, 0]
electrode_matrix['F3']  = [2, 2]
electrode_matrix['FZ']  = [2, 4]
electrode_matrix['F4']  = [2, 6]
electrode_matrix['F8']  = [2, 8]
electrode_matrix['FC5']  = [3, 1]
electrode_matrix['FC1']  = [3, 3]
electrode_matrix['FC2']  = [3, 5]
electrode_matrix['FC6']  = [3, 7]
electrode_matrix['T7']  = [4, 0]
electrode_matrix['C3']  = [4, 2]
electrode_matrix['CZ']  = [4, 4]
electrode_matrix['C4']  = [4, 6]
electrode_matrix['T8']  = [4, 8]
electrode_matrix['CP5']  = [5, 1]
electrode_matrix['CP1']  = [5, 3]
electrode_matrix['CP2']  = [5, 5]
electrode_matrix['CP6']  = [5, 7]
electrode_matrix['P7']  = [6, 0]
electrode_matrix['P3']  = [6, 2]
electrode_matrix['PZ']  = [6, 4]
electrode_matrix['P4']  = [6, 6]
electrode_matrix['P8']  = [6, 8]
electrode_matrix['PO3'] = [7, 3]
electrode_matrix['PO4'] = [7, 5]
electrode_matrix['O1'] = [8, 3]
electrode_matrix['OZ'] = [8, 4]
electrode_matrix['O2'] = [8, 5]

list_electrodes = ['FP1', 'AF3', 'F3', 'F7', 'FC5', 'FC1', 'C3',	'T7',	'CP5',	'CP1',	'P3',	'P7',	'PO3',	'O1',	'OZ',	'PZ',	'FP2',	'AF4', 'FZ', 'F4', 'F8', 'FC6',	'FC2',	'CZ', 'C4', 'T8', 'CP6',	'CP2',	'P4', 	'P8',	'PO4',	'O2']
data_dir = './data_preprocessed_python'
TRAIN_SIZE = 0.9
THRESHOLD = 5

In [ ]:
import glob
import pickle
from collections import Counter

class EmotionDataset(Dataset):
    def __init__ (self, data_dir, type):
       self.data = []
       self.labels = []
       self.cnt = [Counter(), Counter(), Counter(),Counter()]
       #data_dir = './data_preprocessed_python'
       files = glob.glob(os.path.join(data_dir, "*.dat"))
       self.type = type
       #split = int(LEN_RECORD_IN_SECONDS)# *  TRAIN_SIZE)
       
       self.len_files = []
       for file_data in files:
            print(file_data)
            raw_data = pickle.load(open(file_data, 'rb'), encoding='latin1')
            print(raw_data['data'].shape)
            if type == 'train':
                self.data.append(raw_data['data'][0 : int(NVIDEOS * TRAIN_SIZE), :, 3 * NTIMES_IN_SEC :LEN_RECORD_IN_SECONDS * NTIMES_IN_SEC + 3 * NTIMES_IN_SEC])
                self.len_files.append(int(NVIDEOS * TRAIN_SIZE) *  LEN_RECORD_IN_SECONDS - 1)
                self.len_record = LEN_RECORD_IN_SECONDS
            else:
                self.data.append(raw_data['data'][int(NVIDEOS * TRAIN_SIZE) :, :, 3 * NTIMES_IN_SEC :LEN_RECORD_IN_SECONDS * NTIMES_IN_SEC + 3 * NTIMES_IN_SEC])
                self.len_files.append((NVIDEOS  - int(NVIDEOS *TRAIN_SIZE)) * (LEN_RECORD_IN_SECONDS) - 1)
                self.len_record = LEN_RECORD_IN_SECONDS
            labels = raw_data['labels']
            labels = (labels >= THRESHOLD)
            for i in range(4):
              self.cnt[i].update(list(labels[:, i]))
            self.labels.append(labels)
            
       self.len_cumsum = np.cumsum(self.len_files)     
       print(self.data[0].shape)
       print(self.labels[0].shape)



    def __len__(self):
        result =  sum(self.len_files) - 10
        return result

    def get_index_record(self, item):
      for i_file in range(len(self.len_cumsum)):
         #print(item, self.len_cumsum[i_file])
         if (item > self.len_cumsum[i_file]):
            continue
         else:
            break
      if i_file == 0:
         index_in_file = item
      else:
         index_in_file = item  - self.len_cumsum[i_file - 1]
      nvideo = index_in_file//(self.len_record)# * LEN_RECORD_IN_SECONDS *  NTIMES_IN_SEC)
      nsec = (index_in_file - nvideo * self.len_record) # *   NTIMES_IN_SEC)

      return i_file, index_in_file, nvideo, nsec

    def __getitem__(self, item):
      sample = {}
      #print(item)
      i_file, index_in_file, nvideo, nsec = self.get_index_record(item)
      #print(i_file, index_in_file, nvideo, nsec )
      sample['data'] = torch.zeros((HCANALS, WCANALS, NTIMES_IN_SAMPLE))
      for i_canal in range(NCANALS):
        sample_from_one_canal = torch.FloatTensor(self.data[i_file][nvideo, i_canal, nsec * 128 : nsec * 128 + 128])
        #print(sample_from_one_canal.shape)
        sample['data'][electrode_matrix[list_electrodes[i_canal]][0],  electrode_matrix[list_electrodes[i_canal]][1]] = sample_from_one_canal
      if self.type == 'train' :
          sample['labels']  = torch.LongTensor(self.labels[i_file][nvideo])
          #print(nvideo)
      else:
          sample['labels']  = torch.LongTensor(self.labels[i_file][int(NVIDEOS * TRAIN_SIZE) + nvideo])
          #print(int(NVIDEOS * TRAIN_SIZE) + nvideo)    
      #print(sample)
      return sample


In [ ]:
def get_model():
  model = EmotionNet(HCANALS, WCANALS, NFEATURES, NTIMES_IN_SAMPLE).to(device)
  return model


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = get_model()

In [ ]:
# def init_weights(m):
#     for name, param in m.named_parameters():
#         if 'weight' in name:
#             nn.init.normal_(param.data, mean=0, std=0.01)
#         else:
#             nn.init.constant_(param.data, 0)
            
# model.apply(init_weights)

def initialize_weights(m):
    if hasattr(m, 'weight') and m.weight.dim() > 1:
        nn.init.xavier_uniform_(m.weight.data)

model.apply(initialize_weights)

EmotionNet(
  (tfe): Sequential(
    (0): Conv3d(1, 32, kernel_size=(1, 1, 5), stride=(1, 1, 2), padding=(0, 0, 3))
    (1): ReLU()
    (2): Conv3d(32, 32, kernel_size=(1, 1, 3), stride=(1, 1, 2), padding=(0, 0, 1))
    (3): ReLU()
    (4): Conv3d(32, 32, kernel_size=(1, 1, 3), stride=(1, 1, 2), padding=(0, 0, 1))
    (5): ReLU()
    (6): Conv3d(32, 32, kernel_size=(1, 1, 16), stride=(1, 1, 16))
    (7): ReLU()
  )
  (rfe): Sequential(
    (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (1): ReLU()
    (2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (3): ReLU()
  )
  (afe): Asymmetric_feature_extractor(
    (conv): Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1), padding=same)
    (relu): ReLU()
  )
  (flat1): Flatten(start_dim=1, end_dim=3)
  (flat2): Flatten(start_dim=1, end_dim=3)
  (fc1): Linear(in_features=4896, out_features=20, bias=True)
  (relu1): ReLU()
  (drop): Dropout(p=0.3, inplace=False)
  (fc2): Linear(in_features=20, ou

In [ ]:
files = glob.glob(os.path.join(data_dir, "*.dat"))
files.sort()
files = np.asarray(files)
nfiles = len(files)
koeff1 = 0.99
# koeff2 = 0.05
ind_train = random.sample(range(nfiles), int(nfiles * koeff1))
ind_val = list(set(range(nfiles)) - set(ind_train))
#ind_val = random.sample(ind, int(len(ind) * koeff2))
# ind_test = list(set(ind) - set(ind_val))

In [ ]:
print(ind_val)

[26]


In [ ]:
class Args:
  def __init__(self): #(data_path, epoch, batch_siz, image_size, learning_rate, weight_deca, learning_rate, learning_rate_gamma, weight_bce, load, output_dir)
    self.data_path = "/content/drive/MyDrive/MADE/semester2/CV/contest02/data/"
    self.epochs = 2
    self.batch_size = 100
    self.lr= 3e-4
    self.weight_decay= 1e-6
    self.learning_rate=None
    self.learning_rate_gamma=None
    self.weight_bce=1
    self.load=None
    self.output_dir="runs/segmentation_baseline"
    self.data_dir ="./data_preprocessed_python/"# "/content/drive/MyDrive/MADE/Project/train/physionet.org/"
args = Args()    

In [ ]:
train_dataset = EmotionDataset(args.data_dir, 'train')
train_dataloader = DataLoader(train_dataset, batch_size=args.batch_size, num_workers=1,
                              pin_memory=True, shuffle=True, drop_last=True)


val_dataset = EmotionDataset(args.data_dir, 'val')
val_dataloader = DataLoader(val_dataset, batch_size=args.batch_size, num_workers=1,
                              pin_memory=True, shuffle=False, drop_last=False)

In [ ]:
# train_dataset = EmotionDataset(files[ind_train])
# train_dataloader = DataLoader(train_dataset, batch_size=args.batch_size, num_workers=1,
#                               pin_memory=True, shuffle=True, drop_last=True)


# val_dataset = EmotionDataset(files[ind_val])
# val_dataloader = DataLoader(val_dataset, batch_size=args.batch_size, num_workers=1,
#                               pin_memory=True, shuffle=False, drop_last=False)

In [ ]:
criterion = nn.CrossEntropyLoss(reduce = 'mean')#torch.nn.MSELoss()
#optimizer = optim.SGD(model.parameters(), lr=3e-5, momentum = 0.9)#, weight_decay=args.weight_decay)
optimizer = optim.Adam(model.parameters(), lr=3e-4)#, momentum = 0.9)#, weight_decay=args.weight_decay)

/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


In [ ]:
# print(train_dataset.cnt)
# print(val_dataset.cnt)
# print(files[ind_train])
# print(files[ind_val])

In [ ]:
def train(model, loader, criterion, optimizer, device, batch = None):
    model.train()
    train_loss = []
    inputs = []
   
    #lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer)#, mode='min', factor=0.1, patience=10, threshold=0.0001, threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-08, verbose=False)
    for batch in tqdm(loader, total=len(loader), desc="training...", position=0 , leave = True):

            optimizer.zero_grad()
            src  = batch['data'].to(device)
            #print(src.shape)
            trg = batch['labels'][:, 0]

            #print(batch)
            #print(trg.shape)
            levels_pred = model(src)  # B x (2 * NUM_PTS)
            #print(levels_pred.shape)
            levels_pred = levels_pred.cpu()

            #usual cross entropy
            #output = levels_pred[:, 1:].reshape(-1, levels_pred.shape[-1])
            #trg1 = trg[:, 1:].reshape(-1)
            loss = criterion(levels_pred, trg) 

            ##trg1 = trg[:, 1:].reshape(-1)
            ##output = levels_pred[:, 1:].reshape(-1, levels_pred.shape[-1])
            #print(trg1.shape)
            #print(output.shape)
            ##loss = 0
            #print(trg1.shape)
            #print(trg1)
            #print("*********************")
            ##for i in range(OUTPUT_DIM):
              ##  output_class = output[trg1 == i]
              ##  trg_class = trg1[trg1 == i]
                #print(trg_class.shape)
               ## if (trg_class.shape[0] != 0):
                 ##   if (i == 2) or (i == 3):
                   ##    loss += 2 * criterion(output_class, trg_class)/trg_class.shape[0]

            
            #print("after")
            train_loss.append(loss.item())
            loss.backward()
            optimizer.step()
            #break
    return np.mean(train_loss)#, mid_outputs


In [ ]:
def evaluate(model, loader, criterion, device):
    
    model.eval()
    epoch_loss = 0
    history = []
  
    with torch.no_grad():
    
        for s, batch in enumerate(tqdm(loader, total=len(loader), desc="validating...", position=0 , leave = True)):
            src  = batch['data'].to(device)
            #print(src.shape)
            trg = batch['labels'][:, 0]



            levels_pred = model(src)  # B x (2 * NUM_PTS)
            #print(levels_pred.shape)
            levels_pred = levels_pred.cpu()

            #usual cross entropy
            #output = levels_pred[:, 1:].reshape(-1, levels_pred.shape[-1])
            #trg1 = trg[:, 1:].reshape(-1)
            loss = criterion(levels_pred, trg) 

            #trg1 = trg[:, 1:].reshape(-1)
            #output = levels_pred[:, 1:].reshape(-1, levels_pred.shape[-1])
            #print(trg1.shape)
            #print(output.shape)
            #loss = 0
            #print(trg1.shape)
            #print(trg1)
            ##for i in range(OUTPUT_DIM):
              ##  output_class = output[trg1 == i]
              ##  trg_class = trg1[trg1 == i]
                #print(trg_class.shape)
               # if (trg_class.shape[0] != 0):
                    #print(cnt[i], i)
                #    loss += criterion(output_class, trg_class)/trg_class.shape[0]

            epoch_loss += loss.item() 
        
    return epoch_loss / s

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix,classification_report

def calculate_predictions(model, loader):
    model.eval()
    epoch_loss = 0
    history = []
    real = []
    pred = []
    with torch.no_grad():

        for i, batch in enumerate(tqdm(loader, total=len(loader), desc="predicting...", position=0 , leave = True)):
            src  = batch['data'].to(device)
            #print(src.shape)
            trg = batch['labels'][:, 0]
           

            levels_pred = model(src)  # B x (2 * NUM_PTS)
            levels_pred = levels_pred.cpu()
            #print(levels_pred.shape)
            trg_pred = levels_pred.argmax(1)
            
            real.extend(trg)
            pred.extend(trg_pred) 

            
        print(accuracy_score(real, pred)) 
        print(confusion_matrix(real, pred))  
        print(classification_report(real, pred))   
        #plt.hist(real)

In [ ]:
args.epochs = 20000
#criterion =  fnn.mse_loss
train_loss_min = 10000
val_loss_min = 10000
#batch = next(iter(train_dataloader))
for epoch in range(args.epochs):
    #logger.info(f"Starting epoch {epoch + 1}/{args.epochs}.")
    
    train_loss = train(model, train_dataloader, criterion, optimizer ,device)
    #if epoch % 500 == 0:
    print(train_loss)

    if (train_loss < train_loss_min):
        train_loss_min      = train_loss
        torch.save({
                         'model_state_dict': model.state_dict(),
                         'optimizer_state_dict': optimizer.state_dict(),
                       },
                       os.path.join("/content/drive/MyDrive/MADE/Project/RACNN_models/", "train.tgz")
            )  

    val_loss = evaluate(model, val_dataloader, criterion, device)
    # #break
    print(val_loss)

    # #calculate_predictions(model, val_dataloader)
    # if (val_loss < val_loss_min):
    #     val_loss_min      = val_loss
    #     torch.save({'model_state_dict': model.state_dict(),    'optimizer_state_dict': optimizer.state_dict(),}, os.path.join("/content/drive/MyDrive/MADE/Project/RACNN_models/", f"val.tgz"))

training...:   0%|          | 0/690 [00:00<?, ?it/s]

0.6778491351051606


validating...:   0%|          | 0/77 [00:00<?, ?it/s]

0.8143276274204254


training...:   0%|          | 0/690 [00:00<?, ?it/s]

...

0.6375797357248223


validating...:   0%|          | 0/77 [00:00<?, ?it/s]

0.8776344496168589


training...:   0%|          | 0/690 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
calculate_predictions(model, val_dataloader)

predicting...:   0%|          | 0/77 [00:00<?, ?it/s]

0.5145326001571092
[[2644 3029]
 [ 679 1286]]
              precision    recall  f1-score   support

           0       0.80      0.47      0.59      5673
           1       0.30      0.65      0.41      1965

    accuracy                           0.51      7638
   macro avg       0.55      0.56      0.50      7638
weighted avg       0.67      0.51      0.54      7638

